In [4]:
import pysat.solvers
from pysat.card import CardEnc as encode
from random import shuffle
import numpy as np

def bound2(lits, b1, b2, topid):
    # b1 <= n <= b2
    c = encode.atmost(lits = lits, bound = b2, top_id = topid).clauses
    topid = max(max(abs(literal) for clause in c for literal in clause), topid)
    c2 = encode.atleast(lits = lits, bound = b1, top_id = topid).clauses
    topid = max(max(abs(literal) for clause in c2 for literal in clause), topid)

    c.extend(c2)
    return c, topid


for nn in range(4, 30):
    outarr = []
    dim = (nn, nn)
    N = dim[0]*dim[1]
    bbound = 2

    while 1:
        
        v = []
        t = 0
        d = dict()

        for i in range(dim[0]):
            for j in range(dim[1]):
                t += 1
                d[(i,j)] = t

                v.append((i,j))

        clauses = []
        topid = dim[0]*dim[1]+1

        for a,b in v:
            g = d[(a,b)]
            temp = [(a+1,b),(a,b+1),  (a-1,b),(a,b-1)]
            if a == dim[0]-1:
                temp = list(filter(lambda x: x not in ((a+1,b),(a+1,b+1),(a+1,b-1)), temp))


            if b == dim[1]-1:
                temp = list(filter(lambda x: x not in ((a,b+1),(a-1,b+1),(a+1,b+1)), temp))

            if a == 0:
                temp = list(filter(lambda x: x not in ((a-1,b),(a-1,b-1),(a-1,b+1)), temp))

            if b == 0:
                temp = list(filter(lambda x: x not in ((a,b-1),(a+1,b-1),(a-1,b-1)), temp))
                


            #print(temp)

            c = [d[k] for k in temp]
            if len(c) >= 2:
                c = encode.equals(lits = c, bound = 2, top_id = topid).clauses
                #c, topid = bound2(c, 2, 2, topid)

                tempc = []
                for i in c:
                    tempc.append([-g,] + i)
                clauses += tempc
                topid += 4
                #topid = max(max(abs(literal) for clause in clauses for literal in clause), topid)
            else:
                clauses += [[-g,]]
                topid = max(max(abs(literal) for clause in clauses for literal in clause), topid)
        

        #clauses += [[i for i in range(1, dim[0]*dim[1] +1)]]
        clauses += encode.atleast(lits= [i for i in range(1,dim[0]*dim[1] +1)], top_id = topid, bound = bbound).clauses

        solver = pysat.solvers.Cadical()
        for clause in clauses:
            solver.add_clause(clause)


        sat = solver.solve()

        if sat:
            #print( nn, Bound)
            model = solver.get_model()
            temp = [1 if i> 0 else 0 for i in model[:N]]
            bbound = sum(temp)+1
            outarr = temp

        else:
            print(nn, sum(outarr))
            # outarr = temp



            temp = np.array(outarr)
            temp.resize(dim)
            
            import matplotlib.pyplot as plt




            break


ValueError: ignored

In [2]:
!pip install python-sat[pblib,aiger]

     |████████████████████████████████| 1.8 MB 4.6 MB/s 
     |████████████████████████████████| 3.4 MB 33.3 MB/s 
     |████████████████████████████████| 45 kB 3.1 MB/s 
  Created wheel for parsimonious: filename=parsimonious-0.8.1-py3-none-any.whl size=42723 sha256=56c936817029ec76a99d0cafdf32d0a6f08f6a62ea2b3b69a5888b4230fb459f
  Stored in directory: /root/.cache/pip/wheels/88/5d/ba/f27d8af07306b65ee44f9d3f9cadea1db749a421a6db8a99bf
Successfully built parsimonious


In [ ]:
import pysat.solvers
from pysat.card import CardEnc as encode
from random import shuffle
import numpy as np

for nn in range(15, 30):

    dim = (nn, nn)
    N = dim[0]*dim[1]
    

    for bbound in range(N-1, 1, -1):
        
        v = []
        t = 0
        d = dict()

        for i in range(dim[0]):
            for j in range(dim[1]):
                t += 1
                d[(i,j)] = t

                v.append((i,j))

        clauses = []
        topid = dim[0]*dim[1]+1

        for a,b in v:
            g  = d[(a,b)]

            #temp = [(a+1,b),(a,b+1),  (a-1,b),(a,b-1),   (a-1,b-1),(a+1,b+1), (a-1,b+1),(a+1,b-1)]
            temp = [ (a+1,b),(a-1,b), (a,b+1),(a,b-1), ]

            if a == dim[0]-1:
                temp = list(filter(lambda x: x not in ((a+1,b),(a+1,b+1),(a+1,b-1)), temp))

                # temp.remove((a+1,b))
                # temp.remove((a+1,b+1))
                # temp.remove((a+1,b-1))

            if b == dim[1]-1:
                temp = list(filter(lambda x: x not in ((a,b+1),(a-1,b+1),(a+1,b+1)), temp))
                # temp.remove((a,b+1))
                # temp.remove((a-1,b+1))
                # temp.remove((a+1,b+1))

            if a == 0:
                temp = list(filter(lambda x: x not in ((a-1,b),(a-1,b-1),(a-1,b+1)), temp))
                # temp.remove((a-1,b))
                # temp.remove((a-1,b-1))
                # temp.remove((a-1,b+1))

            if b == 0:
                temp = list(filter(lambda x: x not in ((a,b-1),(a+1,b-1),(a-1,b-1)), temp))

                # temp.remove((a,b-1))
                # temp.remove((a+1,b-1))
                # temp.remove((a-1,b-1))
                


            #print(temp)

            c = [d[k] for k in temp]
            
            c = encode.equals(lits = c, bound = 2, top_id = topid).clauses
            tempc = []
            for i in c:
                tempc.append([-g,] + list(map(lambda x:x*(1), i)))
            clauses += tempc
            topid = max(max(abs(literal) for clause in clauses for literal in clause), topid)

        

        #clauses += [[i for i in range(1, dim[0]*dim[1] +1)]]
        clauses += encode.equals(lits= [i for i in range(1,dim[0]*dim[1] +1)], top_id = topid, bound = bbound).clauses

        solver = pysat.solvers.Cadical()
        for clause in clauses:
            solver.add_clause(clause)


        sat = solver.solve()

        if sat:
            
            model = solver.get_model()
            temp = [1 if i> 0 else 0 for i in model[:N]]
            
            print(nn, sum(temp))

            temp = np.array(temp)
            temp.resize(dim)
            
            import matplotlib.pyplot as plt

            # cmap =  ['binary', 'gist_yarg', 'gist_gray', 'gray', 'bone',
            #                     'pink', 'spring', 'summer', 'autumn', 'winter', 'cool',
            #                     'Wistia', 'hot', 'afmhot', 'gist_heat', 'copper']
            cmap = 'copper'
            cmap = 'copper'
            size = 3
            plt.figure(num = None, figsize=(size, size), dpi=300)
            plt.axis('off')
            plot = plt.imshow(temp, cmap = cmap)
            plt.show()
            plt.close()
            break

        else:
            pass

error: ignored

In [2]:
import pysat.solvers
from pysat.card import CardEnc as encode
import numpy as np
from copy import deepcopy

Rule = [[], [2,]]
rulestr = lambda r: 'B' + ''.join(map(str, r[0])) + '/S' + ''.join(map(str, r[1])) + 'V\n'

def m_to_rle(m):
    prefix = 'x = 0, y = 0, rule = ' + rulestr(Rule)
    m = [''.join(list(map(str, i)) + ['$',]) for i in m]
    S = ''.join(m)
    S = S.replace('0', 'b')
    S = S.replace('1', 'o')
    return prefix + S + '!'

for nn in range(2, 30):
    dim = (nn, nn)
    N = dim[0]*dim[1]
    v = []
    t = 0
    d = dict()

    for i in range(dim[0]):
        for j in range(dim[1]):
            t += 1
            d[(i,j)] = t
            v.append((i,j))

    clauses = []
    topid = dim[0]*dim[1]+1

    for a, b in v:
        g  = d[(a,b)]
        temp = [(a+1,b),(a-1,b), (a,b+1),(a,b-1), ]

        if a == dim[0]-1:
            temp = list(filter(lambda x: x not in ((a+1,b),(a+1,b+1),(a+1,b-1)), temp))
        if b == dim[1]-1:
            temp = list(filter(lambda x: x not in ((a,b+1),(a-1,b+1),(a+1,b+1)), temp))
        if a == 0:
            temp = list(filter(lambda x: x not in ((a-1,b),(a-1,b-1),(a-1,b+1)), temp))
        if b == 0:
            temp = list(filter(lambda x: x not in ((a,b-1),(a+1,b-1),(a-1,b-1)), temp))

        tempc = []
        c = [d[k] for k in temp]

        if len(c) == 4:
            tclaus = encode.equals(lits = c, bound = 2, top_id = topid).clauses
            for i in tclaus:
                clauses.append([-g,] +  i)
            topid = max(max(abs(literal) for clause in clauses for literal in clause), topid)
        else:
            clauses.append([-g, ] )
            topid = max(max(abs(literal) for clause in clauses for literal in clause), topid)

    prArr = []
    for bbound in range(N-1, 0, -1):

        clausesTemp = deepcopy(clauses)
        clausesTemp.extend(encode.equals(lits=[i for i in range(1, dim[0]*dim[1]+1)], top_id = topid, bound = bbound).clauses )

        solver = pysat.solvers.Cadical()
        for clause in clausesTemp:
            solver.add_clause(clause)
        sat = solver.solve()

        if sat:
            model = solver.get_model()
            temp = [1 if i> 0 else 0 for i in model[:N]]
            print('nxn: ', nn, 'population: ', sum(temp))
            temp = np.array(temp)
            temp.resize(dim)
            print(m_to_rle(temp.tolist()), '\n')
            break

nxn:  4 population:  4
x = 0, y = 0, rule = B/S2V
bbbb$boob$boob$bbbb$! 

nxn:  5 population:  8
x = 0, y = 0, rule = B/S2V
bbbbb$booob$bobob$booob$bbbbb$! 

nxn:  6 population:  12
x = 0, y = 0, rule = B/S2V
bbbbbb$bbooob$boobob$boboob$booobb$bbbbbb$! 

nxn:  7 population:  16
x = 0, y = 0, rule = B/S2V
bbbbbbb$bboooob$bbobbob$booboob$boboobb$booobbb$bbbbbbb$! 

nxn:  8 population:  24
x = 0, y = 0, rule = B/S2V
bbbbbbbb$booooobb$bobbboob$boboobob$boboobob$bobbboob$booooobb$bbbbbbbb$! 

nxn:  9 population:  32
x = 0, y = 0, rule = B/S2V
bbbbbbbbb$booooooob$bobbbbbob$booooooob$bbbbbbbbb$booooooob$bobbbbbob$booooooob$bbbbbbbbb$! 

nxn:  10 population:  40
x = 0, y = 0, rule = B/S2V
bbbbbbbbbb$boooboooob$bobobobbob$boooboboob$bbbboobobb$bbboobboob$booobbbbob$bobbooobob$boooobooob$bbbbbbbbbb$! 

nxn:  11 population:  52
x = 0, y = 0, rule = B/S2V
bbbbbbbbbbb$bboooobooob$boobbooobob$boboobbbbob$bobooboooob$boobboobbbb$bboboobooob$booboboobob$bobboboboob$boooobooobb$bbbbbbbbbbb$! 

nxn:  12

error: ignored

In [3]:
cc = (4, 8, 12, 16, 24, 32, 40, 52, 64, 76, 90)

cc = [i//4 for i in cc]
cc

[1, 2, 3, 4, 6, 8, 10, 13, 16, 19, 22]